# RAG 多檔案與元數據

本範例展示：
1. **第1個儲存格**：建立多檔案向量資料庫（含元數據）
2. **第2個儲存格**：使用元數據過濾查詢

學習目標：理解如何處理多個來源的文件並使用元數據進行過濾檢索

In [ ]:
# 第1個儲存格：建立多檔案向量資料庫（含元數據）

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
books_dir = os.path.join(current_dir, "books")
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

print(f"書籍目錄: {books_dir}")
print(f"持久化目錄: {persistent_directory}")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保書籍目錄存在
    if not os.path.exists(books_dir):
        raise FileNotFoundError(
            f"目錄 {books_dir} 不存在。請檢查路徑。"
        )

    # 列出目錄中所有文字檔案
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    # 從每個檔案讀取文字內容並儲存元數據
    documents = []
    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path)
        book_docs = loader.load()
        for doc in book_docs:
            # 為每個文件添加元數據以指示其來源
            doc.metadata = {"source": book_file}
            documents.append(doc)

    # 將文件分割成塊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立並持久化向量存儲
    print("\n--- 正在建立並持久化向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- 完成建立並持久化向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

In [ ]:
# 第2個儲存格：使用元數據過濾查詢

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "賈寶玉和林黛玉是什麼關係?"

# 根據查詢檢索相關文件
# search_type="similarity_score_threshold": 使用相似度分數閾值過濾
# k=3: 返回最多 3 個相關文件
# score_threshold=0.1: 較低的閾值，可以返回更多相關文件
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

# 進階查詢：只從特定來源檢索
print("\n--- 只從《紅樓夢》檢索 ---")
# 使用 filter 參數指定元數據條件
specific_docs = db.similarity_search(
    query,
    k=3,
    filter={"source": "紅樓夢.txt"}
)

for i, doc in enumerate(specific_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")